# Convolutional Network
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Reference: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/

https://www.tensorflow.org/get_started/mnist/pros

# Import MNIST data

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [10]:
def weight_variable(shape):
    # add stddev 0.1 to avoide dead neurons
    initial = tf.truncated_normal(shape, stddev=0.1) 
    return tf.Variable(initial)


def bias_variable(shape):
    # due to ReLU, add bias 0.1 to avoide dead neurons
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [14]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer with ReLU 
    h_conv1 = tf.nn.relu(conv2d(x, weights['wc1'], biases['bc1']))
    # Max Pooling (down-sampling)
    h_pool1 = maxpool2d(h_conv1, k=2)

    # Convolution Layer with ReLU 
    h_conv2 = tf.nn.relu(conv2d(h_pool1, weights['wc2'], biases['bc2']))
    # Max Pooling (down-sampling)
    h_pool2 = maxpool2d(h_conv2, k=2)

    # Fully connected layer
    # Reshape h_pool2 output to fit fully connected layer input
    fc1 = tf.reshape(h_pool2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = (tf.add(tf.matmul(fc1, weights['out']), biases['out']))
    
    #out = tf.nn.softmax(tf.matmul(x, W) + b)
    return out

In [15]:
# Store layers weight & bias
# weights = {
#     # 5x5 conv, 1 input, 32 outputs
#     'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
#     # 5x5 conv, 32 inputs, 64 outputs
#     'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
#     # fully connected, 7*7*64 inputs, 1024 outputs
#     'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
#     # 1024 inputs, 10 outputs (class prediction)
#     'out': tf.Variable(tf.random_normal([1024, n_classes]))
# }
weights = {
    # 5x5 conv, 1 input(Channels of image, ex. RGB is 3 input), 
    # 32 outputs: 32 features for each 5x5 patch
    
    'wc1': weight_variable([5, 5, 1, 32]),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': weight_variable([5, 5, 32, 64]),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': weight_variable([7*7*64, 1024]),
    # 1024 inputs, 10 outputs (class prediction)
    'out': weight_variable([1024, n_classes])
}

# biases = {
#     'bc1': tf.Variable(tf.random_normal([32])),
#     'bc2': tf.Variable(tf.random_normal([64])),
#     'bd1': tf.Variable(tf.random_normal([1024])),
#     'out': tf.Variable(tf.random_normal([n_classes]))
# }

biases = {
    'bc1': bias_variable([32]),
    'bc2': bias_variable([64]),
    'bd1': bias_variable([1024]),
    'out': bias_variable([n_classes])
}



# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [16]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 1.577361, Training Accuracy= 0.47656
Iter 2560, Minibatch Loss= 0.672480, Training Accuracy= 0.78906
Iter 3840, Minibatch Loss= 0.309921, Training Accuracy= 0.91406
Iter 5120, Minibatch Loss= 0.284579, Training Accuracy= 0.89062
Iter 6400, Minibatch Loss= 0.186550, Training Accuracy= 0.92969
Iter 7680, Minibatch Loss= 0.246110, Training Accuracy= 0.93750
Iter 8960, Minibatch Loss= 0.165040, Training Accuracy= 0.96094
Iter 10240, Minibatch Loss= 0.213286, Training Accuracy= 0.94531
Iter 11520, Minibatch Loss= 0.113851, Training Accuracy= 0.96875
Iter 12800, Minibatch Loss= 0.083379, Training Accuracy= 0.97656
Iter 14080, Minibatch Loss= 0.115320, Training Accuracy= 0.95312
Iter 15360, Minibatch Loss= 0.175079, Training Accuracy= 0.92969
Iter 16640, Minibatch Loss= 0.075659, Training Accuracy= 0.96875
Iter 17920, Minibatch Loss= 0.151877, Training Accuracy= 0.94531
Iter 19200, Minibatch Loss= 0.098960, Training Accuracy= 0.96094
Iter 20480, Minibatch Loss= 0.06

# Testing Accuracy: 1.0 !!
really amazing